# Homework 3: Regression with Gaussian Processes

------------------------------------------------------
*Machine Learning, Master in Big Data Analytics, 2018-2019*

*Pablo M. Olmos olmos@tsc.uc3m.es*

------------------------------------------------------

The aim of this homework is to solve a real data problem using the Gaussian Process implementation of GPy. The documentation of GPy is avaialable from the [SheffieldML github page](https://github.com/SheffieldML/GPy) or from [this page](http://gpy.readthedocs.org/en/latest/). 

The problem is the prediction of both the heating load (HL) and cooling load (CL) of residential buildings. We consider eight input variables for each building: relative compactness, surface area, wall area, roof area, overall height, orientation, glazing area, glazing area distribution.

In this [paper](https://www.sciencedirect.com/science/article/pii/S037877881200151X) you can find a detailed description of the problem and a solution based on linear regression [(iteratively reweighted least squares (IRLS) algorithm)](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=10&ved=2ahUKEwjZuoLY2OjgAhUs3uAKHUZ7BVcQFjAJegQIAhAC&url=https%3A%2F%2Fpdfs.semanticscholar.org%2F9b92%2F18e7233f4d0b491e1582c893c9a099470a73.pdf&usg=AOvVaw3YDwqZh1xyF626VqfnCM2k) and random forests. Using GPs, our goal is not only estimate accurately both HL and CL, but also get a measure of uncertainty in our predictions.

The data set can be downloaded from the [UCI repository](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency#).

### 1. Loading and preparing the data

* Download the dataset
* Divide at random the dataset into train (80%) and test (20%) datasets 

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

enb = pd.read_excel('ENB2012_data.xlsx')
enb = enb.values

np.random.seed(123)

X=enb[:,0:8]
y=enb[:,8:10]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

((614L, 8L), (614L, 2L))
((154L, 8L), (154L, 2L))


### 2. Setting and optimizing the model

You will train two independent GPs, one to estimate HL and one to estimate CL. For each of the two GPs ...

**On the training data set:**

a) Build a GP regression model based on a RBF kernel with ARD, in which each input dimension is weighted with a different lengthscale. **1.5 points**

b) Fit the covariance function parameters and noise variance. **1 point** 

c) According to the ARD parameters found, what variables are more important for the regression? Compare it to Table 8 in this [paper](https://www.sciencedirect.com/science/article/pii/S037877881200151X) **1.5 points**

**On the test data set:**

d) Compute the test mean absolute error error and the test mean square error (MSE)  using the GP posterior mean and the optimized hyperparameters. Compare your results with Tables 6 and 7 in this [paper](https://www.sciencedirect.com/science/article/pii/S037877881200151X).
**1.5 points**

2) Try to improve your results by using a more complicated kernel, in which you combine various covariance functions. In this [link](http://nbviewer.jupyter.org/github/SheffieldML/notebook/blob/master/GPy/basic_kernels.ipynb) you can see how to define different kernels and combine  them. Comments the results. **2 points**



## 1A & B) 

In [3]:
import GPy
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

%matplotlib inline
%config InlineBackend.figure_format = 'retina'   ##QUALITY FIGURES!!
plt.rcParams["figure.figsize"] = [8,8]

In [5]:
# define kernel
ker = GPy.kern.RBF(input_dim=8, ARD=True) 

# create simple GP model for each variable
m = GPy.models.GPRegression(X_train,y_train,ker)

In [8]:
display(m)

GP_regression.,value,constraints,priors
rbf.variance,1.0,+ve,
rbf.lengthscale,"(8L,)",+ve,
Gaussian_noise.variance,1.0,+ve,


In [11]:
m.optimize(messages=True, max_f_eval = 1000)

SEJveChjaGlsZHJlbj0oVkJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgbWF4PTEwMDApLCBIVE1MKHZhbHVlPXUnJykpKSwgQm94KGNoaWxkcmVuPShIVE1MKHZhbHVlPXUnJynigKY=


In [13]:
print("kernel Lengthscale:")
print("")
print(ker.lengthscale.values)

kernel Lengthscale:

[  1.           1.           1.           1.           1.
 234.14905259   0.32181893 264.10068573]


# 1C)

# 1D)

In [29]:
nsamples = 100

#Generate predictions from the model
posteriortest = m.posterior_samples_f(X_test, full_cov=True, size=nsamples)
meantest = m.predict(X_test, full_cov=False)

In [28]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

print("Y1 Mean Absolute Error: " + str(mean_absolute_error(y1_test, meantest[0][:,0])))
print("Y1 Mean Squared Error: " + str(mean_squared_error(y1_test, meantest[0][:,0])))

print("Y2 Mean Absolute Error: " + str(mean_absolute_error(y2_test, meantest[0][:,1])))
print("Y2 Mean Squared Error: " + str(mean_squared_error(y2_test, meantest[0][:,1])))


Y1 Mean Absolute Error: 0.34413247326073054
Y1 Mean Squared Error: 0.222068776142243
Y2 Mean Absolute Error: 0.9339193544970908
Y2 Mean Squared Error: 2.112545075277692


# 2)

In [35]:
ker_combined = GPy.kern.RBF(input_dim=8, ARD=True) + GPy.kern.White(8) + GPy.kern.Matern32(8)

# create simple GP model for each variable
m_improved = GPy.models.GPRegression(X_train,y_train,ker_combined)

m_improved.optimize(messages=False, max_f_eval = 1000)

meanImprovedtest = m_improved.predict(X_test, full_cov=False)

print("Y1 Mean Absolute Error: " + str(mean_absolute_error(y1_test, meanImprovedtest[0][:,0])))
print("Y1 Mean Squared Error: " + str(mean_squared_error(y1_test, meanImprovedtest[0][:,0])))

print("Y2 Mean Absolute Error: " + str(mean_absolute_error(y2_test, meanImprovedtest[0][:,1])))
print("Y2 Mean Squared Error: " + str(mean_squared_error(y2_test, meanImprovedtest[0][:,1])))


Y1 Mean Absolute Error: 0.3183214614892159
Y1 Mean Squared Error: 0.2636062882688694
Y2 Mean Absolute Error: 0.41294483496577733
Y2 Mean Squared Error: 0.3285022890175988


In [36]:
ker_combined = GPy.kern.RBF(input_dim=8, ARD=True) * GPy.kern.White(8) * GPy.kern.Matern32(8)

# create simple GP model for each variable
m_improved = GPy.models.GPRegression(X_train,y_train,ker_combined)

m_improved.optimize(messages=False, max_f_eval = 1000)

meanImprovedtest = m_improved.predict(X_test, full_cov=False)


print("Y1 Mean Absolute Error: " + str(mean_absolute_error(y1_test, meanImprovedtest[0][:,0])))
print("Y1 Mean Squared Error: " + str(mean_squared_error(y1_test, meanImprovedtest[0][:,0])))

print("Y2 Mean Absolute Error: " + str(mean_absolute_error(y2_test, meanImprovedtest[0][:,1])))
print("Y2 Mean Squared Error: " + str(mean_squared_error(y2_test, meanImprovedtest[0][:,1])))


Y1 Mean Absolute Error: 20.982272727272726
Y1 Mean Squared Error: 526.4627837662338
Y2 Mean Absolute Error: 23.318376623376622
Y2 Mean Squared Error: 616.558175974026


In [37]:
ker_combined = GPy.kern.RBF(input_dim=8, ARD=True) + GPy.kern.White(8) + GPy.kern.Matern32(8) + GPy.kern.Exponential(8)

# create simple GP model for each variable
m_improved = GPy.models.GPRegression(X_train,y_train,ker_combined)

m_improved.optimize(messages=False, max_f_eval = 1000)

meanImprovedtest = m_improved.predict(X_test, full_cov=False)

print("Y1 Mean Absolute Error: " + str(mean_absolute_error(y1_test, meanImprovedtest[0][:,0])))
print("Y1 Mean Squared Error: " + str(mean_squared_error(y1_test, meanImprovedtest[0][:,0])))

print("Y2 Mean Absolute Error: " + str(mean_absolute_error(y2_test, meanImprovedtest[0][:,1])))
print("Y2 Mean Squared Error: " + str(mean_squared_error(y2_test, meanImprovedtest[0][:,1])))


 C:\Users\niall\Anaconda2\lib\site-packages\GPy\kern\src\stationary.py:137: RuntimeWarning:overflow encountered in square
 C:\Users\niall\Anaconda2\lib\site-packages\GPy\kern\src\stationary.py:168: RuntimeWarning:overflow encountered in divide
 C:\Users\niall\Anaconda2\lib\site-packages\GPy\kern\src\stationary.py:454: RuntimeWarning:overflow encountered in multiply
 C:\Users\niall\Anaconda2\lib\site-packages\GPy\kern\src\stationary.py:454: RuntimeWarning:overflow encountered in exp
 C:\Users\niall\Anaconda2\lib\site-packages\GPy\kern\src\stationary.py:454: RuntimeWarning:invalid value encountered in multiply


Y1 Mean Absolute Error: 0.3479557572364571
Y1 Mean Squared Error: 0.3086263190704844
Y2 Mean Absolute Error: 0.4406707757816722
Y2 Mean Squared Error: 0.36590540886003614


In [ ]:
ker_combined = GPy.kern.RBF(input_dim=8, ARD=True) + GPy.kern.White(8) + GPy.kern.Exponential(8)

# create simple GP model for each variable
m_improved = GPy.models.GPRegression(X_train,y_train,ker_combined)

m_improved.optimize(messages=False, max_f_eval = 1000)

meanImprovedtest = m_improved.predict(X_test, full_cov=False)

print("Y1 Mean Absolute Error: " + str(mean_absolute_error(y1_test, meanImprovedtest[0][:,0])))
print("Y1 Mean Squared Error: " + str(mean_squared_error(y1_test, meanImprovedtest[0][:,0])))

print("Y2 Mean Absolute Error: " + str(mean_absolute_error(y2_test, meanImprovedtest[0][:,1])))
print("Y2 Mean Squared Error: " + str(mean_squared_error(y2_test, meanImprovedtest[0][:,1])))


### Sparse GP implementation 

Try to implement an sparse version of the GP regressor, optimized to find a set of **inducing points** that the GP relies on to do the prediction. Measure the test error prediction for 20, 40, and 100 inducing points. **2.5 points**

In [32]:
search_space = [20,40,100]

for i in search_space:
    m_sp = GPy.models.SparseGPRegression(X_train, y_train, num_inducing=i, kernel=ker)
    m_sp.optimize(messages=False, max_f_eval = 1000)
    meantest_sp = m_sp.predict(X_test, full_cov=False)

    print("Y1 Mean Absolute Error: " + str(mean_absolute_error(y1_test, meantest_sp[0][:,0])))
    print("Y1 Mean Squared Error: " + str(mean_squared_error(y1_test, meantest_sp[0][:,0])))

    print("Y2 Mean Absolute Error: " + str(mean_absolute_error(y2_test, meantest_sp[0][:,1])))
    print("Y2 Mean Squared Error: " + str(mean_squared_error(y2_test, meantest_sp[0][:,1])))
    
    print("")


Y1 Mean Absolute Error: 5.816672544200181
Y1 Mean Squared Error: 93.17450616112286
Y2 Mean Absolute Error: 5.9682847154536
Y2 Mean Squared Error: 113.6507607582618

Y1 Mean Absolute Error: 1.4400630714376288
Y1 Mean Squared Error: 5.104248694700504
Y2 Mean Absolute Error: 1.4300563904781334
Y2 Mean Squared Error: 4.216733578155614



 C:\Users\niall\Anaconda2\lib\site-packages\paramz\transformations.py:111: RuntimeWarning:overflow encountered in expm1
 C:\Users\niall\Anaconda2\lib\site-packages\paramz\transformations.py:111: RuntimeWarning:invalid value encountered in expm1


Y1 Mean Absolute Error: 0.6693802153271133
Y1 Mean Squared Error: 0.8179704748808401
Y2 Mean Absolute Error: 1.0054031055057004
Y2 Mean Squared Error: 2.2771548101861567

